# Titanic

This is a quick response to some thoughts I had on [Chris Deotte's](https://www.kaggle.com/cdeotte) great kernel [Titanic using Name only](https://www.kaggle.com/cdeotte/titanic-using-name-only-0-81818/notebook).

To sumarise quickly, his approach makes predictions based on the following rules:

* All males die except boys in families where all females and boys live.

* All females live except those in families where all females and boys die.

Boys are defined as passengers with the title "Master", and families are defined as groups of people with the same surname.

I was wondering what would happen if you group passengers sharing same ticket number, instead of passengers with the same surname. My main thought process for this was:

* Passengers with the same surname may not come from the same family (could come from multiple families).

* There may be important non-surname based groups, for example travelling friends, non-married couples etc.  

So this notebook uses Chris' approach with ticket grouping and compares the results.

In [1]:
# load the data
import pandas as pd
df = pd.read_csv('train.csv',index_col='PassengerId')

In [2]:
# select females and masters (boys)
boy = (df.Name.str.contains('Master')) | ((df.Sex=='male') & (df.Age<13))
female = df.Sex=='female'
boy_or_female = boy | female

# no. females + boys on ticket
n_ticket = df[boy_or_female].groupby('Ticket').Survived.count()

# survival rate amongst females + boys on ticket
tick_surv = df[boy_or_female].groupby('Ticket').Survived.mean()

In [3]:
# function to create relevant features for test data
def create_features(frame):
    frame['Boy'] = (frame.Name.str.contains('Master')) | ((frame.Sex=='male') & (frame.Age<13))
    
    frame['Female'] = (frame.Sex=='female').astype(int)

    # if ticket exists in training data, fill NTicket with no. women+boys
    # on that ticket in the training data.
    frame['NTicket'] = frame.Ticket.replace(n_ticket)
    # otherwise NTicket=0
    frame.loc[~frame.Ticket.isin(n_ticket.index),'NTicket']=0

    # if ticket exists in training data, fill TicketSurv with
    # women+boys survival rate in training data  
    frame['TicketSurv'] = frame.Ticket.replace(tick_surv)
    # otherwise TicketSurv=0
    frame.loc[~frame.Ticket.isin(tick_surv.index),'TicketSurv']=0

    # return data frame only including features needed for prediction
    return frame[['Female','Boy','NTicket','TicketSurv']]


In [4]:
# predict survival for a passenger
def did_survive(row):
    if row.Female:
        # predict died if all women+boys on ticket died
        if (row.NTicket>0) and (row.TicketSurv==0):
            return 0
        # predict survived for all other women
        else:
            return 1
        
    elif row.Boy:
        # predict survived if all women+boys on ticket survived
        if (row.NTicket>0) and (row.TicketSurv==1):
            return 1
        # predict died for all other boys
        else:
            return 0
        
    else:
        # predict all men die
        return 0

In [5]:
# load test data
df_test = pd.read_csv('test.csv',index_col='PassengerId')

# extract the features to use
X = create_features(df_test)

# predict test data
pred = X.apply(did_survive,axis=1)

# create submission file
pred = pd.DataFrame(pred) 
pred.rename(columns={0:'Survived'},inplace=True)
pred.to_csv('submission.csv')

print(pred.Survived.value_counts())

0    265
1    153
Name: Survived, dtype: int64


# Results

This approach gives me a leaderboard score of 0.813 compared to Chris Deotte's score of 0.823, so is marginally worse than the surname based approach.

By comparing with his output file, I see that I predict three passengers in the test data survive, whilst Chris predicts they die. These three passengers are displayed below:

In [12]:
# passengers where I predict differently
# (based on comparison with Chris Deotte's output file)
id_nomatch = [910,929,1172]

# display these passengers
display(df_test.loc[id_nomatch])

,Roberts,Deotte,Match
PassengerId,,,
910,1,0,False
929,1,0,False
1172,1,0,False


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Boy,Female,NTicket,TicketSurv
PassengerId,,,,,,,,,,,,,,
910,3,"Ilmakangas, Miss. Ida Livija",female,27.0,1,0,STON/O2. 3101270,7.9250,NaN,S,False,1,0,0
929,3,"Cacic, Miss. Manda",female,21.0,0,0,315087,8.6625,NaN,S,False,1,0,0
1172,3,"Oreskovic, Miss. Jelka",female,23.0,0,0,315085,8.6625,NaN,S,False,1,0,0


And the passengers in the training data with surnames matching the three test data passengers above are:

In [14]:
# families in training data with member in test data that I predict differently
display(df.loc[df.Name.str.contains('Ilmakangas')])
display(df.loc[df.Name.str.contains('Cacic')])
display(df.loc[df.Name.str.contains('Oreskovic')])

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
730,0,3,"Ilmakangas, Miss. Pieta Sofia",female,25.0,1,0,STON/O2. 3101271,7.925,NaN,S


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
472,0,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,NaN,S
535,0,3,"Cacic, Miss. Marija",female,30.0,0,0,315084,8.6625,NaN,S


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
405,0,3,"Oreskovic, Miss. Marija",female,20.0,0,0,315096,8.6625,NaN,S
726,0,3,"Oreskovic, Mr. Luka",male,20.0,0,0,315094,8.6625,NaN,S


The Ilmakangas appear to be adult sisters travelling together (indicated by SibSp=1), but who bought tickets separately. But the Cacics and Oreskovics have no siblings, spouses, children or parents onboard according to the data. However, they all travel in the same class, with the same fare and with similar ticket numbers. They may be unrelated, but are likely to be travelling cousins/similar who bought tickets separately.

In summary, whilst I was hoping grouping by ticket would catch a greater variety of groups I'd missed the point that groups of friends etc. are very likely to have bought tickets separately. So only grouping by ticket appears to miss a few relationships, rather than catch more.